In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install face_recognition

In [ ]:
# SKLearn Implemention
from sklearn.metrics import log_loss
log_loss(["REAL", "FAKE", "FAKE", "REAL"],
         [[.1, .9], [.9, .1], [.8, .2], [.35, .65]])

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import cv2
plt.style.use('ggplot')
from IPython.display import Video
from IPython.display import HTML
import os

In [ ]:
train_sample_metadata = pd.read_json('/kaggle/input/deepfake-detection-challenge/train_sample_videos/metadata.json').T
train_sample_metadata.head()

In [ ]:
train_sample_metadata.groupby('label')['label'].count().plot(figsize=(15, 5), kind='bar', title='Distribution of Labels in the Training Set')
plt.show()

In [ ]:
import cv2 as cv
import os
import matplotlib.pylab as plt
train_dir = '/kaggle/input/deepfake-detection-challenge/train_sample_videos/'
fig, ax = plt.subplots(1,1, figsize=(15, 15))
train_video_files = [train_dir + x for x in os.listdir(train_dir)]
# video_file = train_video_files[30]
video_file = '/kaggle/input/deepfake-detection-challenge/train_sample_videos/akxoopqjqz.mp4'
cap = cv.VideoCapture(video_file)
success, image = cap.read()
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
cap.release()   
ax.imshow(image)
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
ax.title.set_text(f"FRAME 0: {video_file.split('/')[-1]}")
plt.grid(False)

In [ ]:
import face_recognition
face_locations = face_recognition.face_locations(image)

# https://github.com/ageitgey/face_recognition/blob/master/examples/find_faces_in_picture.py
from PIL import Image

print("I found {} face(s) in this photograph.".format(len(face_locations)))

for face_location in face_locations:

    # Print the location of each face in this image
    top, right, bottom, left = face_location
    print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))

    # You can access the actual face itself like this:
    face_image = image[top:bottom, left:right]
    fig, ax = plt.subplots(1,1, figsize=(5, 5))
    plt.grid(False)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.imshow(face_image)

In [ ]:
face_landmarks_list = face_recognition.face_landmarks(image)

In [ ]:
# https://github.com/ageitgey/face_recognition/blob/master/examples/find_facial_features_in_picture.py
# face_landmarks_list
from PIL import Image, ImageDraw
pil_image = Image.fromarray(image)
d = ImageDraw.Draw(pil_image)

for face_landmarks in face_landmarks_list:

    # Print the location of each facial feature in this image
    for facial_feature in face_landmarks.keys():
        print("The {} in this face has the following points: {}".format(facial_feature, face_landmarks[facial_feature]))

    # Let's trace out each facial feature in the image with a line!
    for facial_feature in face_landmarks.keys():
        d.line(face_landmarks[facial_feature], width=3)

# Show the picture
display(pil_image)

In [ ]:
os.makedirs('/kaggle/frames')

os.chdir('/kaggle/frames/')

os.makedirs('/kaggle/frames/real')

os.makedirs('/kaggle/frames/fake')

In [ ]:
# Change the data set here
for fn,row in train_sample_metadata.iterrows():
    video_file = f'/kaggle/input/deepfake-detection-challenge/train_sample_videos/{fn}'
    cap = cv2.VideoCapture(video_file)

    frames = []
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            frames.append(frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    
    padding = 40
    for i in [0, 25, 50, 75, 100, 125, 150, 175, 250, 275]:
        try:
            frame = frames[i]
        except:
            continue
        #fig, ax = plt.subplots(1,1, figsize=(5, 5))
        face_locations = face_recognition.face_locations(frame)
        if len(face_locations) == 0:
            print(f'Count find face in frame {i}')
            continue
        top, right, bottom, left = face_locations[0]
        frame_face = frame[top-padding:bottom+padding, left-padding:right+padding]
        face_landmarks_list = face_recognition.face_landmarks(frame_face)
        if len(face_landmarks_list) == 0:
            print(f'Could not identify face landmarks for frame {i}')
            continue
        face_landmarks = face_landmarks_list[0]
        pil_image = Image.fromarray(frame_face)
        d = ImageDraw.Draw(pil_image)
        landmark_face_array = np.array(pil_image)
#         image = cv.cvtColor(landmark_face_array, cv.COLOR_BGR2RGB)
        if row['label'] == "FAKE":
            os.chdir('/kaggle/frames/fake')
        else:
            os.chdir('/kaggle/frames/real')
        cv.imwrite(fn+str(i)+".jpg", landmark_face_array)

In [ ]:
print(os.listdir('/kaggle/frames/real/'))

In [ ]:
print(len(os.listdir('/kaggle/frames/fake/')))

In [ ]:
Image.open('/kaggle/frames/real/egghxjjmfg.mp4150.jpg')